# Task 3

Implement a PMMH algorithm, which relies on the bootstrap filter constructed in Step 1, and compare its performance to the MCMC sampler of step 2. For instance, what is the impact of the number of particles N on the mixing properties of PMMH, compared the *ideal* algorithm developped in Step 2 ? (This can be assessed by plotting the ACFs for instance; see also the discussion in the course on calibrating PMMH.) 